# Akeed Restaurant Recommendation Challenge

## Importing libraries:

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import psycopg2
import geopandas as gpd
from shapely import wkt

## Inspecting the base layers:

### Base Layer 1

In [52]:
#Training Locations
training_locations = pd.read_csv("./train_locations.csv")
print("\n This table has " + str(len(training_locations)) + " rows.")
training_locations.head()
#training_locations.isnull().sum()


 This table has 59503 rows.


,customer_id,location_number,location_type,latitude,longitude
0,02SFNJH,0,NaN,1.682392,-78.789737
1,02SFNJH,1,NaN,1.679137,0.766823
2,02SFNJH,2,NaN,-0.498648,0.661241
3,RU43CXC,0,Home,0.100853,0.438165
4,BDFBPRD,0,NaN,2.523125,0.733464


### Base Layer 2

In [74]:
#Training Customers
training_customers = pd.read_csv("./train_customers.csv")
print("\n This table has " + str(len(training_customers)) + " rows.")
training_customers.head()
#training_customers.isnull().sum()


 This table has 34674 rows.


,akeed_customer_id,gender,dob,status,verified,language,created_at,updated_at
0,TCHWPBT,Male,NaN,1,1,EN,2018-02-07 19:16:23,2018-02-07 19:16:23
1,ZGFSYCZ,Male,NaN,1,1,EN,2018-02-09 12:04:42,2018-02-09 12:04:41
2,S2ALZFL,Male,NaN,0,1,EN,2018-03-14 18:31:43,2018-03-14 18:31:42
3,952DBJQ,Male,NaN,1,1,EN,2018-03-15 19:47:07,2018-03-15 19:47:07
4,1IX6FXS,Male,NaN,1,1,EN,2018-03-15 19:57:01,2018-03-15 19:57:01


### Base Layer 3

In [73]:
#Orders
orders = pd.read_csv("./orders.csv")
print("\n This table has " + str(len(orders)) + " rows.")
orders.head()
#orders.isnull().sum()
#orders.iloc[:,18:21]


 This table has 135303 rows.


/home/kusasa/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (15,16,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,...,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163238.0,92PEE24,1.0,7.6,2,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,105,2019-08-01 05:30:16,0,NaN,92PEE24 X 0 X 105
1,163240.0,QS68UD8,1.0,8.7,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,294,2019-08-01 05:31:10,0,Work,QS68UD8 X 0 X 294
2,163241.0,MB7VY5F,2.0,14.4,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:31:33,0,NaN,MB7VY5F X 0 X 83
3,163244.0,KDJ951Y,1.0,7.1,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,90,2019-08-01 05:34:54,0,Home,KDJ951Y X 0 X 90
4,163245.0,BAL0RVT,4.0,27.2,1,NaN,0.0,NaN,NaN,No,...,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,83,2019-08-01 05:35:51,0,Work,BAL0RVT X 0 X 83


### Base Layer 4

In [58]:
#Vendors
vendors = pd.read_csv("./vendors.csv")
print("\n This table has " + str(len(vendors)) + " rows.")
vendors.head()
#vendors.isnull().sum()


 This table has 100 rows.


,id,authentication_id,latitude,longitude,vendor_category_en,vendor_category_id,delivery_charge,serving_distance,is_open,OpeningTime,...,open_close_flags,vendor_tag,vendor_tag_name,one_click_vendor,country_id,city_id,created_at,updated_at,device_type,display_orders
0,4,118597.0,-0.588596,0.754434,Restaurants,2.0,0.0,6.0,1.0,11:00AM-11:30PM,...,1.0,"2,4,5,8,91,22,12,24,16,23","Arabic,Breakfast,Burgers,Desserts,Free Deliver...",Y,1.0,1.0,2018-01-30 14:42:04,2020-04-07 15:12:43,3,1
1,13,118608.0,-0.471654,0.744470,Restaurants,2.0,0.7,5.0,1.0,08:30AM-10:30PM,...,1.0,"4,41,51,34,27,15,24,16,28","Breakfast,Cakes,Crepes,Italian,Pasta,Pizzas,Sa...",Y,1.0,1.0,2018-05-03 12:32:06,2020-04-05 20:46:03,3,1
2,20,118616.0,-0.407527,0.643681,Restaurants,2.0,0.0,8.0,1.0,08:00AM-10:45PM,...,1.0,"4,8,91,10","Breakfast,Desserts,Free Delivery,Indian",Y,1.0,1.0,2018-05-04 22:28:22,2020-04-07 16:35:55,3,1
3,23,118619.0,-0.585385,0.753811,Restaurants,2.0,0.0,5.0,1.0,10:59AM-10:30PM,...,1.0,"5,8,30,24","Burgers,Desserts,Fries,Salads",Y,1.0,1.0,2018-05-06 19:20:48,2020-04-02 00:56:17,3,1
4,28,118624.0,0.480602,0.552850,Restaurants,2.0,0.7,15.0,1.0,11:00AM-11:45PM,...,1.0,5,Burgers,Y,1.0,1.0,2018-05-17 22:12:38,2020-04-05 15:57:41,3,1


## Pre-process tables where need be

### Deal with null values

In [ ]:
#Training Locations


In [ ]:
#Orders table


### Deal with mixed data-type fields

In [ ]:
#Orders table


### Scale field values where needed

### Add valuable calculated fields

In [ ]:
#Add geometry field to the training locations table
training_locations['geom']=""
row_no = 0
for row in training_locations.iterrows():
    training_locations.loc[row_no, 'geom'] = ['POINT(' + str(row[1]['longitude']) + ' ' + str(row[1]['latitude']) + ')']
    row_no += 1
training_locations['geom'] = training_locations['geom'].apply(wkt.loads)
training_locations_gdf = gpd.GeoDataFrame(training_locations, geometry='geom', crs='epsg:4326')

In [ ]:
training_locations_gdf.head

## Exploring Relationships using plots

### Exploration 1

In [ ]:
#Spatial Relations

ax = training_locations_gdf.plot(edgecolor = 'grey', facecolor = 'none', figsize = (18, 12), legend=True)
#roads.plot(ax=ax, color = 'purple')
#ax.set(xlim=(15, 35), ylim=(-37.5, -20))
ax.set_axis_off()
plt.title(label = 'Training Locations', fontweight = 'bold')
plt.tight_layout()

### Exploration 2

In [ ]:
# Linear Relations



## Prediction Modelling